In [23]:
import pandas as pd
import math

#csv to pandas
df = pd.read_csv('new_dropped_parsing_Kyiv.csv')


subway_locations = pd.read_excel('metro stations.xlsx')
subway_locations.columns = ['name', 'lat', 'lng']

subway_locations_dict = subway_locations.to_dict('records')

print(subway_locations_dict)


# test_df = df.head(10)

#iterate over dataframe
for index, row in df.iterrows():
    latitude = row.latitude
    longitude = row.longitude

    closest_station = None
    closest_distance = float('inf')  # Set initial distance to infinity

    for subway_station in subway_locations_dict:
        lat = subway_station['lat']
        lng = subway_station['lng']

        # Calculate distance between given location and current subway station
        d_lat = math.radians(lat - latitude)
        d_lng = math.radians(lng - longitude)
        a = math.sin(d_lat / 2) ** 2 + math.cos(math.radians(latitude)) * math.cos(math.radians(lat)) * math.sin(d_lng / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = 6371 * c * 1000  # 6371 km is the approximate radius of the Earth, multiplied by 1000 to convert to meters

        # Update closest station if current station is closer
        if distance < closest_distance:
            closest_station = subway_station
            closest_distance = distance

    if closest_station:
        name = closest_station['name']
        location = f'{closest_station["lat"]}, {closest_station["lng"]}'
        df.at[index, "closest_subway"] = name
        df.at[index, "subway_location"] = location

        df.to_csv('rieltor_subways_new2.csv', index=False)
    else:
        print('No subway stations found within the specified radius.')

[{'name': 'Житомирська', 'lat': 50.456175, 'lng': 30.365628}, {'name': 'Святошин', 'lat': 50.457903, 'lng': 30.390614}, {'name': 'Нивки', 'lat': 50.458653, 'lng': 30.404042}, {'name': 'Берестейська', 'lat': 50.458333, 'lng': 30.420833}, {'name': 'Шулявська', 'lat': 50.455, 'lng': 30.445278}, {'name': 'Політехнічний інститут', 'lat': 50.450833, 'lng': 30.466111}, {'name': 'Вокзальна', 'lat': 50.441667, 'lng': 30.4875}, {'name': 'Університет', 'lat': 50.444167, 'lng': 30.505833}, {'name': 'Театральна', 'lat': 50.444722, 'lng': 30.518056}, {'name': 'Хрещатик', 'lat': 50.44726, 'lng': 30.52483}, {'name': 'Арсенальна', 'lat': 50.443889, 'lng': 30.545556}, {'name': 'Дніпро', 'lat': 50.441111, 'lng': 30.558611}, {'name': 'Гідропарк', 'lat': 50.445556, 'lng': 30.576944}, {'name': 'Лівобережна', 'lat': 50.451389, 'lng': 30.598889}, {'name': 'Дарниця', 'lat': 50.455556, 'lng': 30.613056}, {'name': 'Чернігівська', 'lat': 50.459722, 'lng': 30.630833}, {'name': 'Лісова', 'lat': 50.464639, 'lng': 30

In [29]:

df = pd.read_csv('rieltor_subways_new2.csv')

def haversine(lat1, lon1, subway):
    if type(subway) == float:
        return 0
    """
    Calculate the great circle distance between two points
    on the Earth's surface using their latitude and longitude
    coordinates. Returns the distance in kilometers.
    """
    R = 6371  # Radius of the Earth in kilometers
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    # convert to float
    
    lat2_rad = math.radians(float(subway.split(',')[0]))
    lon2_rad = math.radians(float(subway.split(',')[1]))
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    a = math.sin(delta_lat/2) * math.sin(delta_lat/2) + \
        math.cos(lat1_rad) * math.cos(lat2_rad) * \
        math.sin(delta_lon/2) * math.sin(delta_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return distance

# Calculate distance to nearest subway station for each row
df['distance_to_subway'] = df.apply(lambda row: haversine(row['latitude'], row['longitude'], row['subway_location']), axis=1)

# Assign values to a new column based on distance thresholds
df['distance_category'] = pd.cut(df['distance_to_subway'], bins=[0, 2, 5, float('inf')], labels=[1, 2, 3], right=False)

# Convert the new column to integer data type
df['distance_category'] = df['distance_category'].astype(int)

df.to_csv('rieltor_subways_new2.csv', index=False)

